<a href="https://colab.research.google.com/github/joat26/NLP/blob/main/Weather_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT LIBRARIES


In [20]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [21]:
API_KEY= '2c00adc9b036ce87b1701367d0de1777'
BASE_URL= 'https://api.openweathermap.org/data/2.5/'

1. Fetch

In [77]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url)
  data = response.json()
  return{
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0]['description'],
      'country': data['sys']['country'],
      'wind_gust_dir': data[ 'wind']['deg'],
      'pressure': data[ 'main'] ['pressure'],
      'wind_gust_speed': data[ 'wind']['speed']
  }

In [40]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.drop_duplicates()
  return df

In [41]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])


  X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
  y = data['RainTomorrow']
  return X, y, le

TRAIN RAIN PREDICTION MODEL

In [54]:
def train_rain_model(X, y):

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  model = RandomForestClassifier(n_estimators=100, random_state=42)

  model.fit(X_train, y_train) #train the model

  y_pred = model.predict(X_test) #to make predictions on test set

  print("Mean Squared Error for Rain Model")

  print (mean_squared_error (y_test, y_pred))

  return model

In [43]:
def prepare_regression_data(data, feature):

  X,y=[], []



  for i in range(len(data) - 1):

    X.append(data[feature].iloc[i])

    y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1, 1)

  y = np.array(y)

  return X, y

In [44]:
def train_regression_model(X, y):

  model = RandomForestRegressor(n_estimators=100, random_state=42)

  model.fit(X, y)

  return model

In [45]:
def predict_future(model, current_value):

  predictions=[current_value]

  for i in range(5):

    next_value=model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])

  return predictions[1:]

  for i in range(5):

    next_value=model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])

  return predictions [1:]

In [79]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from time import strftime
from dataclasses import replace


def weather_view():
    city = input('Enter any city name: ')
    current_weather = get_current_weather(city)


# Load historical data
    historical_data = read_historical_data('weather.csv')



#prepae and train the rain prediction model


    X, y, le = prepare_data(historical_data)

    rain_model = train_rain_model(X, y)

# Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
      ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75), ("N", 348.75, 360)
]

    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

# Prepare current data for prediction
    current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['wind_gust_speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp'],
}

    current_df = pd.DataFrame([current_data])
    rain_prediction = rain_model.predict(current_df)[0]

# Train regression models for temperature and humidity
    X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(X_temp, y_temp)
    hum_model = train_regression_model(X_hum, y_hum)

# Predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

# Generate future time labels
    timezone = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

# Display current weather information
    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}°C")
    print(f"Temp Min: {current_weather['temp_min']}°C")
    print(f"Temp Max: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

# Display future predictions
    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

# Visualize the data
weather_view()


Enter any city name: tokyo
Mean Squared Error for Rain Model
0.16216216216216217
City: tokyo, JP
Current Temperature: 8°C
Feels Like: 6°C
Temp Min: 7°C
Temp Max: 9°C
Humidity: 54%
Weather Prediction: broken clouds
Rain Prediction: No

Future Temperature Predictions:
17:00: 8.7°C
18:00: 10.8°C
19:00: 12.8°C
20:00: 12.9°C
21:00: 11.8°C

Future Humidity Predictions:
17:00: 58.6%
18:00: 67.1%
19:00: 76.5%
20:00: 62.0%
21:00: 56.4%
